In [1]:
import pandas as pd
import numpy as np

# •	Extraction

In [2]:
# 1. Extraction:
df= pd.read_csv("pension_dataset_20000.csv")

In [3]:
df.head()

,pensioner_id,name,dob,gender,join_date,retirement_date,years_of_service,monthly_pension,pension_type,lump_sum,region,last_contribution,spouse_count,beneficiary_name,payment_method,bank_account_mask,inflation_index,active_flag,created_at
0,P000001,Michael Martin,1973-12-22,M,1997-04-16,2030-07-10,17,735.61,Defined Benefit,0.00,West,2029-08-14,1,James Lopez,Bank Transfer,XXXX-XXXX-7396,1.0151,1,2023-11-11T17:10:25.588933
1,P000002,Thomas Martinez,1936-11-29,F,1968-05-29,2005-09-29,25,1756.99,Defined Benefit,0.00,South,2004-03-16,0,NaN,Bank Transfer,XXXX-XXXX-8513,1.0197,0,2024-05-12T17:10:25.589190
2,P000003,Mary Martin,1944-11-18,M,1974-04-03,2005-12-07,16,1042.74,Defined Contribution,0.00,East,2005-05-16,0,NaN,Bank Transfer,XXXX-XXXX-5297,1.0159,0,2023-03-08T17:10:25.589500
3,P000004,Susan Thomas,1948-05-21,M,1974-05-09,2018-04-06,23,1844.33,Defined Benefit,0.00,North,2018-02-13,0,James Lopez,Bank Transfer,XXXX-XXXX-6258,1.0209,0,2021-09-27T17:10:25.589788
4,P000005,Thomas Davis,1964-11-20,F,1987-03-19,2024-12-15,42,2172.51,Defined Benefit,60830.28,East,2024-11-18,1,John Smith,Bank Transfer,XXXX-XXXX-9110,1.0298,0,2021-11-03T17:10:25.590267


# •	Cleaning & Standardization


In [4]:
# checking null values
df.isnull().sum()


pensioner_id            0
name                    0
dob                     0
gender                  0
join_date               0
retirement_date         0
years_of_service        0
monthly_pension         0
pension_type            0
lump_sum                0
region                  0
last_contribution       0
spouse_count            0
beneficiary_name     8014
payment_method          0
bank_account_mask       0
inflation_index         0
active_flag             0
created_at              0
dtype: int64

In [17]:
# Convert to datetime
df["dob"] = pd.to_datetime(df["dob"], errors="coerce")
df["retirement_date"] = pd.to_datetime(df["retirement_date"], errors="coerce")
df["join_date"] = pd.to_datetime(df["join_date"], errors="coerce")


In [18]:
# Remove duplicate records (if any)
df = df.drop_duplicates()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   pensioner_id       20000 non-null  object        
 1   name               20000 non-null  object        
 2   dob                20000 non-null  datetime64[ns]
 3   gender             20000 non-null  object        
 4   join_date          20000 non-null  datetime64[ns]
 5   retirement_date    20000 non-null  datetime64[ns]
 6   years_of_service   20000 non-null  int64         
 7   monthly_pension    20000 non-null  float64       
 8   pension_type       20000 non-null  object        
 9   lump_sum           20000 non-null  float64       
 10  region             20000 non-null  object        
 11  last_contribution  20000 non-null  object        
 12  spouse_count       20000 non-null  int64         
 13  beneficiary_name   11986 non-null  object        
 14  paymen

In [19]:
df.dtypes

pensioner_id                 object
name                         object
dob                  datetime64[ns]
gender                       object
join_date            datetime64[ns]
retirement_date      datetime64[ns]
years_of_service              int64
monthly_pension             float64
pension_type                 object
lump_sum                    float64
region                       object
last_contribution            object
spouse_count                  int64
beneficiary_name             object
payment_method               object
bank_account_mask            object
inflation_index             float64
active_flag                   int64
created_at                   object
age_at_retirement           float64
pension_status               object
dtype: object

In [20]:
df.columns = ( df.columns .str.strip() .str.lower() .str.replace(" ", "_") .str.replace(r"[^\w_]", "", regex=True) )

# •	Data Consistency & Derivation

In [21]:
# Age at retirement
# Calculated only for retirees with valid dates
df["age_at_retirement"] = (df["retirement_date"] - df["dob"]).dt.days.div(365.25).round(1)
    
df["age_at_retirement"]


0        56.5
1        68.8
2        61.1
3        69.9
4        60.1
         ... 
19995    65.5
19996    64.3
19997    65.9
19998    68.7
19999    56.0
Name: age_at_retirement, Length: 20000, dtype: float64

In [22]:
# Pension status
df["pension_status"] = df["retirement_date"].apply(
        lambda x: "Retired" if pd.notna(x) else "Active"
    )


In [23]:
df.head()

,pensioner_id,name,dob,gender,join_date,retirement_date,years_of_service,monthly_pension,pension_type,lump_sum,...,last_contribution,spouse_count,beneficiary_name,payment_method,bank_account_mask,inflation_index,active_flag,created_at,age_at_retirement,pension_status
0,P000001,Michael Martin,1973-12-22,M,1997-04-16,2030-07-10,17,735.61,Defined Benefit,0.00,...,2029-08-14,1,James Lopez,Bank Transfer,XXXX-XXXX-7396,1.0151,1,2023-11-11T17:10:25.588933,56.5,Retired
1,P000002,Thomas Martinez,1936-11-29,F,1968-05-29,2005-09-29,25,1756.99,Defined Benefit,0.00,...,2004-03-16,0,NaN,Bank Transfer,XXXX-XXXX-8513,1.0197,0,2024-05-12T17:10:25.589190,68.8,Retired
2,P000003,Mary Martin,1944-11-18,M,1974-04-03,2005-12-07,16,1042.74,Defined Contribution,0.00,...,2005-05-16,0,NaN,Bank Transfer,XXXX-XXXX-5297,1.0159,0,2023-03-08T17:10:25.589500,61.1,Retired
3,P000004,Susan Thomas,1948-05-21,M,1974-05-09,2018-04-06,23,1844.33,Defined Benefit,0.00,...,2018-02-13,0,James Lopez,Bank Transfer,XXXX-XXXX-6258,1.0209,0,2021-09-27T17:10:25.589788,69.9,Retired
4,P000005,Thomas Davis,1964-11-20,F,1987-03-19,2024-12-15,42,2172.51,Defined Benefit,60830.28,...,2024-11-18,1,John Smith,Bank Transfer,XXXX-XXXX-9110,1.0298,0,2021-11-03T17:10:25.590267,60.1,Retired


In [24]:
#Re-calculating the years_of_service field
df["years_of_service"] = (df["retirement_date"] - df["join_date"]) .dt.days.div(365.25).round(1)
df["years_of_service"]      

0        33.2
1        37.3
2        31.7
3        43.9
4        37.7
         ... 
19995    41.9
19996    44.8
19997    43.0
19998    49.5
19999    35.8
Name: years_of_service, Length: 20000, dtype: float64

In [25]:
# Masking sensitive data
df["beneficiary_name"] = df["beneficiary_name"].apply(
        lambda x: "MASKED" if pd.notna(x) else pd.NA
    )

df["beneficiary_name"].head()

0    MASKED
1      <NA>
2      <NA>
3    MASKED
4    MASKED
Name: beneficiary_name, dtype: object

In [26]:
# Save to CSV for MySQL import
csv_path = "cleaned_pensions.csv"
df.to_csv(csv_path, index=False)

print("CSV file created for MySQL import:", csv_path)

CSV file created for MySQL import: cleaned_pensions.csv


In [59]:
#Follwing code are to transfer cleaned file to SQL

from sqlalchemy import create_engine
# MySQL credentials

username = 'root'

password = '123456789'

host = 'localhost'       

port = '3306'            # default MySQL port

database = 'Pension_database'


# Creating the SQLAlchemy engine

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')


# Read CSV file

#df = pd.read_csv('D:\sabiha\study\Fortray\workshop\assignment_pension\pension_dataset_20000.csv')


# To Import the data into MySQL 

df.to_sql(
    name="pension_records",
    con=engine,
    if_exists="replace",
    index=False
)


print("Data imported successfully!")


Data imported successfully!


In [60]:
import os
print(os.getcwd())

C:\Users\User
